# Handling irradiation scenarios

## Nuclides and Pulses

The basic units on which F4Enix relies to operate with irradiation scenarios are the ``Nuclide`` and ``Pulse`` classes. A Nuclide only mandatory attribute is the zaid but it can support much more information.

In [2]:
from f4enix.input.irradiation import Nuclide

nuclide = Nuclide(1001)
print(nuclide.write_to_formula())
print(nuclide.write_to_int_string())

H1
1001


Additional data includes library (for parent isotopes of d1suned), metastable flags and IRS flags.
Special flags conventions:

| Flag         | Formula tag    | Int tag    |
|--------------|----------------|------------|
| IRS          | `irs<XX>`      | `999<XX>`  |
| metastable   | `<XX>m`        | `<XX>900`  |

In [3]:
nuclide = Nuclide(1001, lib="91c", metastable=True, IRS_active=True)
print(nuclide.write_to_formula())
print(nuclide.write_to_int_string())

irsH1m.91c
9991001900.91c


A ``Nuclide`` can also be read from the same strings that can produce

In [4]:
print(Nuclide.from_formula("Mo99m.80c"))
print(Nuclide.from_int_string("1001.91c"))

Mo99m.80c
H1.91c


A ``Pulse`` instead represents a basic pulse in an irradiation scenario. It is defined by
an intensity and a time interval that can be expressed in different units.

In [5]:
from f4enix.input.irradiation import Pulse
from f4enix.constants import TIME_UNITS

pulse = Pulse(2, 2.5e10, TIME_UNITS.MONTH)
print(pulse)
print(f'seconds: {pulse.time}') # internally the time is always stored in seconds

Pulse(time=2.0 TIME_UNITS.MONTH, intensity=25000000000.0)
seconds: 5259666.666666667


## Irradiation Scenario

Starting from a collection of pulses, an irradiation scenario can be built.

In [6]:
from f4enix.input.irradiation import IrradiationScenario

pulses = (
    [Pulse(1, 1e10, TIME_UNITS.DAY), Pulse(2, 0, TIME_UNITS.DAY)]*4 +
    [Pulse(1, 1e11, TIME_UNITS.HOUR)] +
    [Pulse(2, 5e10, TIME_UNITS.HOUR)]*3
)

irr_scenario = IrradiationScenario(pulses)
irr_scenario.pulses

[Pulse(time=1.0 TIME_UNITS.DAY, intensity=10000000000.0),
 Pulse(time=2.0 TIME_UNITS.DAY, intensity=0),
 Pulse(time=1.0 TIME_UNITS.DAY, intensity=10000000000.0),
 Pulse(time=2.0 TIME_UNITS.DAY, intensity=0),
 Pulse(time=1.0 TIME_UNITS.DAY, intensity=10000000000.0),
 Pulse(time=2.0 TIME_UNITS.DAY, intensity=0),
 Pulse(time=1.0 TIME_UNITS.DAY, intensity=10000000000.0),
 Pulse(time=2.0 TIME_UNITS.DAY, intensity=0),
 Pulse(time=1.0 TIME_UNITS.HOUR, intensity=100000000000.0),
 Pulse(time=2.0 TIME_UNITS.HOUR, intensity=50000000000.0),
 Pulse(time=2.0 TIME_UNITS.HOUR, intensity=50000000000.0),
 Pulse(time=2.0 TIME_UNITS.HOUR, intensity=50000000000.0)]

Additionally, the ``IrradiationScenario`` class can also be initialized from an existing
FISPACT-II input or legacy d1stime script input with the methods:

```python
IrradiationScenario.from_legacy_d1stime('path_to_file')
```

and

```python
IrradiationScenario.from_fispact('path_to_file')
```

By default, a unique cooling time at 0s is added to the irradiation scenario.
This can be changed using the ``set_cooling_times()`` method that allows to define times relative
to each other or absolute.

In [10]:
# times relative to each other
irr_scenario.set_cooling_times([(1, TIME_UNITS.SECOND), (3, TIME_UNITS.SECOND)], absolute=False)
print('Relative times:')
print(irr_scenario.cooling_times)
print(irr_scenario.cooling_labels) # always expressed as absolute times

irr_scenario.set_cooling_times([(1, TIME_UNITS.SECOND), (3, TIME_UNITS.SECOND)])

print('\nAbsolute times:')
print(irr_scenario.cooling_times)
print(irr_scenario.cooling_labels) # always expressed as absolute times

Relative times:
[Pulse(time=1.0 TIME_UNITS.SECOND, intensity=0.0), Pulse(time=3.0 TIME_UNITS.SECOND, intensity=0.0)]
['1.0s', '4s']

Absolute times:
[Pulse(time=1.0 TIME_UNITS.SECOND, intensity=0.0), Pulse(time=2.0 TIME_UNITS.SECOND, intensity=0.0)]
['1s', '3s']


## Time correction factors calculation

When using a Direct 1 Step (D1S) approach for the calculation of shut down dose rates, time correction factors are required. These factors are computed recursively at each irradiation pulse and for each Nuclide as:

$$
C_0 = 0
$$

$$
C_{i} = \frac{I_i}{\mathrm{norm}} \cdot (1-e^{-\lambda \Delta t_i}) + C_{i-1} \cdot e^{-\lambda \Delta t_i}
$$

where:
- $\lambda$ is the decay constant of the radioactive nuclide
- $\Delta t_i$ is the duration of the $i$-th pulse
- $I$ is the intensity of the neutron source at the $i$-th pulse
- $\mathrm{norm}$ is an arbitrary normalization factor

F4Enix allows to compute these factors with the ``TCF_Computer`` class. Decay constants are computed with data taken from [decay2020 decay library](https://www.oecd-nea.org/dbdata/fispact/decay2020.tar.bz2) downloadable from the FISPACT website. These values can be recalled with ad hoc get methods:

In [9]:
from f4enix.input.irradiation import TCF_Computer

tcf_computer = TCF_Computer()  # mainly loads the decay data

# Get a lambda or half life value for a single nuclide
nuclide = Nuclide.from_formula("Co60")
lambda_val = tcf_computer.get_lambda(nuclide)
half_life_val = tcf_computer.get_half_life(nuclide)

print(f'Lambda for Co-60: {lambda_val} 1/s')
print(f'Half-life for Co-60: {half_life_val} s')

Lambda for Co-60: 4.167050502344595e-09 1/s
Half-life for Co-60: 166340000.0 s


While the time correction factors can be computed for different isotopes
at the same time. Factors will be computed at each cooling time specified in the
irradiation scenario.

In [14]:
nuclides = [Nuclide.from_formula("Co60"), Nuclide.from_formula("Co60m")]

tcf_results = tcf_computer.compute_correction_factors(irr_scenario, nuclides, norm=1e10)
tcf_results

array([[2.03637835e-03, 4.99448453e+00],
       [2.03637834e-03, 4.98347184e+00]])

In [15]:
# if needed, it can be organized as a DataFrame
import pandas as pd
df = pd.DataFrame(tcf_results)
df.columns = [nuclide.write_to_formula() for nuclide in nuclides]
df.index = irr_scenario.cooling_labels
df

,Co60,Co60m
1s,0.002036,4.994485
3s,0.002036,4.983472
